В Pandas мы не можем извлечь из столбцов, содержащий в себе некоторый текст, извлечь определенные слова, даты или время, но можно сделать это с помощью пользовательских функций. Т.е. можно написать функцию, которая принимает на вход какой-либо элемент, обрабатывает его и возвращает результат, и применить ее ко всем элементам с помощью метода apply(). В результате будет возвращен объект Series

Рассмотрим пример: у нас есть столбец с адресами объектов недвижимости, проблема в том, что почти все они уникальны, убедимся в этом,  вычислив количество уникальных значений для столбца

In [44]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_ps.csv', sep=',')

In [45]:
print(melb_df['Address'].nunique())

13378


Для прогноза цены объекта такое большое количество категорий может плохо сказаться при построении модели. Говорят, что такой признак не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку

Из-за этого зависимость между целевым признаком, который мы хотим предсказать и признаками, на основе которых мы делаем предсказание, становится очень сложной. Точность моделирования снизится, а производительность резко упадет

Обычно такие признаки удаляют, но можно сделать иначе: извлечем из признака адреса характеристику подтипа улицы(улица, шоссе, авеню, бульвар)

Адрес строится следующим образом: сначала указывается номер дома и корпус, затем название улицы, а в конце - подтип улицы, иногда к подтипу добавляется географическая отметка(N - север, S - юг и т.д.) она нам не нужна. Для выделения подтипа улицы напишем функцию:

In [31]:
# На вход функции поступает строка с адресом
def get_street_type(address):
    # Создаем список географических пометок exclude_list
    exclude_list = ['N', 'S', 'W', 'E']
    # Методом split() разбиваем строку на слова по пробелу
    # В результате получаем список слов в строке и заносим его в переменную address_list
    address_list = address.split()
    # Обрезаем список, оставляя на нем только последний элемент, потенциальный подтип улицы и заносим в street_type
    street_type = address_list[-1]
    # делаем проверку на то, что полученный подтип является географической пометкой
    # Для этого проверяем его на наличие в списке exclude_list
    if street_type in exclude_list:
        # Если переменная является географической пометкой,
        # переопределяем ее на второй элемент с конца списка address_list
        street_type = address_list[-2]
    return street_type


Теперь применим ее к столбцу с адресом. Для этого передадим функцию get_street_type в агрумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types

## функция пишется для одного элемента столбца, а метод apply() применяет ее ко всем элементам. Используемая функция обязательно должна иметь возвращаемое значение. 

In [32]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [33]:
# У нас получилось сократить число уникальных значений до 56

print(street_types.nunique())

56


In [34]:
# У нас есть 56 уникальных значений. Однако результат можно улучшить. Для начала посмотрим на частоту
# каждого подтипа улицы с помощью value_counts()
display(street_types.value_counts())

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Grn             4
Fairway         4
Mews            4
Gdns            4
Righi           3
Crossway        3
Esp             2
Ridge           2
Victoria        2
Crofts          2
Athol           1
Highway         1
Cove            1
Grange          1
Res             1
Terrace         1
Qy              1
Glade           1
Nook            1
Eyrie           1
Loop            1
Dell            1
East            1
Summit          1
Grand           1
Gra             1
Hts             1
Outlook         1
Woodland        1
Ave             1
Co

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов резко падает
Применим очень распространенный метод уменьшения количества уникальных категорий - выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'others'
Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10,
то есть мы хотим отобрать 10 наиболее популярных подтипов. Извлечем их названия с помощью атрибута index, а результат занесем в переменную popular_stypes

In [35]:
popular_stypes = street_types.value_counts().nlargest(10).index
display(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')

In [36]:
street_types = street_types.apply(lambda x: 'Av' if x == 'Avenue' else x)
street_types = street_types.apply(lambda x: 'Bvd' if x == 'Boulevard' else x)
street_types = street_types.apply(lambda x: 'Pde' if x == 'Parade' else x)

Теперь у нас есть список наиболее популярных подтипов улиц, введем lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать х, в противном случае она будет возвращать строку "other". Наконец, применим строку к Series street_types, полученной ранее, а результат определим в новый столбец StreetType

In [37]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [38]:
# посмотрим на результатирующее число уникальных подтипов
print(melb_df['StreetType'].nunique())

11


In [46]:
day = melb_df['WeekDaySale'].value_counts()
def get_weekkend(day):
    if day in [5,6]:
        return 1
    else:
        return 0
    

KeyError: 'WeekDaySale'

In [31]:
melf_df['WeekEnd'] = melf_df['WeekDaySale'].apply(get_weekkend)

KeyError: 'WeekDaySale'

In [47]:
popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
# заменяем имена компаний, не попавших в список популярных, на строку 'other'
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other') 

a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
print(round(a/b, 1))

1.3
